Load Data

In [26]:
import pandas as pd
# Remove the limit on the number of columns displayed
pd.set_option('display.max_columns', None)

In [147]:
import requests
import pandas as pd

# Your API token
API_KEY = 'cadf4921727a4fc98eb9e50304e95e1f'
BASE_URL = 'https://api.football-data.org/v4/'

# Function to fetch data from the API
def fetch_data(endpoint, params=None):
    headers = {'X-Auth-Token': API_KEY}
    response = requests.get(BASE_URL + endpoint, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()
    elif response.status_code == 403:
        print("Access forbidden: Check if your token is valid for v4.")
    elif response.status_code == 429:
        print("Rate limit exceeded: Try again later.")
    else:
        print(f"Failed to fetch data: {response.status_code}")
        print(response.text)
    return None

# Fetch Premier League matches for the current season
def get_premier_league_matches():
    matches_data = fetch_data('competitions/PL/matches')
    if matches_data and 'matches' in matches_data:
        # Parse matches into a DataFrame
        matches = matches_data['matches']
        matches_df = pd.DataFrame([
            {
                'Match ID': match['id'],
                'Season': match['season']['startDate'][:4] + '/' + match['season']['endDate'][:4],
                'Date': match['utcDate'],
                'Home Team': match['homeTeam']['name'],
                'Away Team': match['awayTeam']['name'],
                'Home Score': match['score']['fullTime']['home'],
                'Away Score': match['score']['fullTime']['away'],
                'Winner': match['score']['winner']
            } for match in matches
        ])
        return matches_df
    else:
        print("No match data available.")
        return None

# Fetch and display data
matches_df = get_premier_league_matches()

# Display data and save locally
if matches_df is not None:
    print("Premier League Matches Data:")
    display(matches_df.tail())
    matches_df.to_csv("premier_league_matches.csv", index=False)
    print("Data saved to 'premier_league_matches_result.csv'")
else:
    print("No data fetched.")


Premier League Matches Data:


,Match ID,Season,Date,Home Team,Away Team,Home Score,Away Score,Winner
375,497785,2024/2025,2025-05-25T15:00:00Z,Newcastle United FC,Everton FC,NaN,NaN,None
376,497786,2024/2025,2025-05-25T15:00:00Z,Nottingham Forest FC,Chelsea FC,NaN,NaN,None
377,497787,2024/2025,2025-05-25T15:00:00Z,Southampton FC,Arsenal FC,NaN,NaN,None
378,497788,2024/2025,2025-05-25T15:00:00Z,Tottenham Hotspur FC,Brighton & Hove Albion FC,NaN,NaN,None
379,497789,2024/2025,2025-05-25T15:00:00Z,Wolverhampton Wanderers FC,Brentford FC,NaN,NaN,None


Data saved to 'premier_league_matches_result.csv'


In [9]:
matches_df.columns 

Index(['Match ID', 'Season', 'Date', 'Home Team', 'Away Team', 'Home Score',
       'Away Score', 'Winner'],
      dtype='object')

In [216]:
import requests
import pandas as pd

# Function to fetch league table data from football-data.org
def fetch_league_table(season):
    API_URL = "https://api.football-data.org/v4/competitions/PL/standings"
    HEADERS = {"X-Auth-Token": 'cadf4921727a4fc98eb9e50304e95e1f'}  # Replace with your actual API key
    
    params = {"season": season}
    response = requests.get(API_URL, headers=HEADERS, params=params)
    
    if response.status_code == 200:
        data = response.json()
        standings = data["standings"][0]["table"]  # Assuming we're fetching the overall table
        # Extract relevant details
        table = [
            {
                "Position": team["position"],
                "Team": team["team"]["name"],
                "Played Games": team["playedGames"],
                "Won": team["won"],
                "Drawn": team["draw"],
                "Lost": team["lost"],
                "Goals For": team["goalsFor"],
                "Goals Against": team["goalsAgainst"],
                "Goal Difference": team["goalDifference"],
                "Points": team["points"]
            }
            for team in standings
        ]
        return pd.DataFrame(table)
    else:
        print(f"Failed to fetch data for season {season}. Status code: {response.status_code}")
        return None

# Fetch data for both seasons
season_2023 = 2023
season_2024 = 2024

table_2023 = fetch_league_table(season_2023)
table_2024 = fetch_league_table(season_2024)

# Save data to CSV
if table_2023 is not None:
    csv_file_2023 = "Premier_League_Table_2023_2024.csv"
    table_2023.to_csv(csv_file_2023, index=False)
    print(f"Premier League Table for Season 2023/2024 saved to {csv_file_2023}.")
    
if table_2024 is not None:
    csv_file_2024 = "Premier_League_Table_2024_2025.csv"
    table_2024.to_csv(csv_file_2024, index=False)
    print(f"Premier League Table for Season 2024/2025 saved to {csv_file_2024}.")


Premier League Table for Season 2023/2024 saved to Premier_League_Table_2023_2024.csv.
Premier League Table for Season 2024/2025 saved to Premier_League_Table_2024_2025.csv.


In [218]:
table_2024.head()

,Position,Team,Played Games,Won,Drawn,Lost,Goals For,Goals Against,Goal Difference,Points
0,1,Liverpool FC,14,11,2,1,29,11,18,35
1,2,Chelsea FC,15,9,4,2,35,18,17,31
2,3,Arsenal FC,15,8,5,2,29,15,14,29
3,4,Manchester City FC,15,8,3,4,27,21,6,27
4,5,Nottingham Forest FC,15,7,4,4,19,18,1,25


In [221]:
len(table_2024)

20

One gameweek before before the current standing

In [92]:
table_2024.head()

,Position,Team,Played Games,Won,Drawn,Lost,Goals For,Goals Against,Goal Difference,Points
0,1,Liverpool FC,13,11,1,1,26,8,18,34
1,2,Arsenal FC,13,7,4,2,26,14,12,25
2,3,Chelsea FC,13,7,4,2,26,14,12,25
3,4,Brighton & Hove Albion FC,13,6,5,2,22,17,5,23
4,5,Manchester City FC,13,7,2,4,22,19,3,23


In [40]:
matches_df.columns

Index(['Match ID', 'Season', 'Date', 'Home Team', 'Away Team', 'Home Score',
       'Away Score', 'Winner'],
      dtype='object')

In [8]:
import pandas as pd
season2023_df = pd.read_csv("season2023.csv")

In [12]:
print(season2023_df.head())

   Unnamed: 0        Date   Time            Comp        Round  Day Venue  \
0           1  2023-08-11  20:00  Premier League  Matchweek 1  Fri  Away   
1           3  2023-08-19  20:00  Premier League  Matchweek 2  Sat  Home   
2           4  2023-08-27  14:00  Premier League  Matchweek 3  Sun  Away   
3           5  2023-09-02  15:00  Premier League  Matchweek 4  Sat  Home   
4           6  2023-09-16  15:00  Premier League  Matchweek 5  Sat  Away   

  Result  GF  GA  ...  Match Report  Notes    Sh   SoT  Dist   FK PK PKatt  \
0      W   3   0  ...  Match Report    NaN  17.0   8.0  13.9  0.0  0     0   
1      W   1   0  ...  Match Report    NaN  14.0   4.0  17.9  0.0  0     0   
2      W   2   1  ...  Match Report    NaN  29.0   9.0  17.3  2.0  0     1   
3      W   5   1  ...  Match Report    NaN   6.0   4.0  14.8  0.0  1     1   
4      W   3   1  ...  Match Report    NaN  29.0  13.0  16.4  1.0  0     0   

  Season            Team  
0   2024  ManchesterCity  
1   2024  Manchester

In [14]:
season2023_df.columns

Index(['Unnamed: 0', 'Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result',
       'GF', 'GA', 'Opponent', 'xG', 'xGA', 'Poss', 'Attendance', 'Captain',
       'Formation', 'Referee', 'Match Report', 'Notes', 'Sh', 'SoT', 'Dist',
       'FK', 'PK', 'PKatt', 'Season', 'Team'],
      dtype='object')

# Scrape from fbref

## Define the URL of the FBref Page Containing the Table:


In [133]:
url = 'https://fbref.com/en/comps/9/2024-2025/schedule/2024-2025-Premier-League-Scores-and-Fixtures'

## Read the HTML Tables from the URL:

In [135]:
# Use pandas to read all tables from the webpage
tables = pd.read_html(url)


## Identify and Extract the Desired Table:

In [159]:
# Assuming the desired table is the first one on the page
df = tables[0]

In [161]:
df.head(5)

,Wk,Day,Date,Time,Home,xG,Score,xG.1,Away,Attendance,Venue,Referee,Match Report,Notes
0,1.0,Fri,2024-08-16,20:00,Manchester Utd,2.4,1–0,0.4,Fulham,73297.0,Old Trafford,Robert Jones,Match Report,NaN
1,1.0,Sat,2024-08-17,12:30,Ipswich Town,0.5,0–2,2.6,Liverpool,30014.0,Portman Road Stadium,Tim Robinson,Match Report,NaN
2,1.0,Sat,2024-08-17,15:00,Newcastle Utd,0.3,1–0,1.8,Southampton,52196.0,St James' Park,Craig Pawson,Match Report,NaN
3,1.0,Sat,2024-08-17,15:00,Nott'ham Forest,1.3,1–1,1.2,Bournemouth,29763.0,The City Ground,Michael Oliver,Match Report,NaN
4,1.0,Sat,2024-08-17,15:00,Everton,0.5,0–3,1.4,Brighton,39217.0,Goodison Park,Simon Hooper,Match Report,NaN


In [163]:
df.columns

Index(['Wk', 'Day', 'Date', 'Time', 'Home', 'xG', 'Score', 'xG.1', 'Away',
       'Attendance', 'Venue', 'Referee', 'Match Report', 'Notes'],
      dtype='object')

In [165]:
df.describe()

,Wk,xG,xG.1,Attendance,Notes
count,380.000000,126.000000,126.000000,122.000000,0.0
mean,19.500000,1.605556,1.320635,40170.942623,NaN
std,10.980313,0.850629,0.769552,16700.999350,NaN
min,1.000000,0.100000,0.000000,11161.000000,NaN
25%,10.000000,1.000000,0.800000,29885.000000,NaN
50%,19.500000,1.550000,1.200000,35359.500000,NaN
75%,29.000000,2.100000,1.600000,53114.500000,NaN
max,38.000000,4.400000,4.400000,73829.000000,NaN


In [167]:
# Save the DataFrame to a CSV file
df.to_csv('premier_league_2023_2024_results_fbref.csv', index=False)

In [169]:
import pandas as pd

# URL of the Premier League 2024-2025 Scores & Fixtures page
url = 'https://fbref.com/en/comps/9/schedule/Premier-League-Scores-and-Fixtures'

# Read all tables from the webpage
tables = pd.read_html(url)

# The desired table is the first one
df = tables[0]

# Display the first few rows of the DataFrame
print(df.head())


    Wk  Day        Date   Time             Home   xG Score  xG.1         Away  \
0  1.0  Fri  2024-08-16  20:00   Manchester Utd  2.4   1–0   0.4       Fulham   
1  1.0  Sat  2024-08-17  12:30     Ipswich Town  0.5   0–2   2.6    Liverpool   
2  1.0  Sat  2024-08-17  15:00    Newcastle Utd  0.3   1–0   1.8  Southampton   
3  1.0  Sat  2024-08-17  15:00  Nott'ham Forest  1.3   1–1   1.2  Bournemouth   
4  1.0  Sat  2024-08-17  15:00          Everton  0.5   0–3   1.4     Brighton   

   Attendance                 Venue         Referee  Match Report  Notes  
0     73297.0          Old Trafford    Robert Jones  Match Report    NaN  
1     30014.0  Portman Road Stadium    Tim Robinson  Match Report    NaN  
2     52196.0        St James' Park    Craig Pawson  Match Report    NaN  
3     29763.0       The City Ground  Michael Oliver  Match Report    NaN  
4     39217.0         Goodison Park    Simon Hooper  Match Report    NaN  


In [171]:
df.describe()

,Wk,xG,xG.1,Attendance,Notes
count,380.000000,126.000000,126.000000,122.000000,0.0
mean,19.500000,1.605556,1.320635,40170.942623,NaN
std,10.980313,0.850629,0.769552,16700.999350,NaN
min,1.000000,0.100000,0.000000,11161.000000,NaN
25%,10.000000,1.000000,0.800000,29885.000000,NaN
50%,19.500000,1.550000,1.200000,35359.500000,NaN
75%,29.000000,2.100000,1.600000,53114.500000,NaN
max,38.000000,4.400000,4.400000,73829.000000,NaN


In [48]:
len(df)

417

In [50]:
df.columns

Index(['Wk', 'Day', 'Date', 'Time', 'Home', 'xG', 'Score', 'xG.1', 'Away',
       'Attendance', 'Venue', 'Referee', 'Match Report', 'Notes'],
      dtype='object')

In [175]:
df.to_csv("premier_league_2024_2025_results_fbref.csv", index=False)

## Odds data

In [131]:
API_KEY = 'ffb20d24ac64dccb79042ce2d0ab6e87'

In [177]:
import pandas as pd
import requests

# Premier League sport key
API_KEY = 'ffb20d24ac64dccb79042ce2d0ab6e87'
SPORT_KEY = 'soccer_epl'

# Base API URLs
base_url = 'https://api.the-odds-api.com/v4/sports'
event_odds_url = 'https://api.the-odds-api.com/v4/events/{eventId}/odds'

# Parameters for initial odds fetch
params = {
    'apiKey': API_KEY,
    'regions': 'uk',
    'markets': 'h2h',  # Fetch basic h2h odds initially
    'oddsFormat': 'decimal'
}

# Step 1: Fetch matches
response = requests.get(f'{base_url}/{SPORT_KEY}/odds', params=params)

if response.status_code == 200:
    data = response.json()
    matches_data = []

    for match in data:
        match_info = {
            'Match ID': match['id'],
            'Date': match['commence_time'],
            'Home Team': match['home_team'],
            'Away Team': match['away_team']
        }

        # Extract h2h odds
        for bookmaker in match.get('bookmakers', []):
            for market in bookmaker.get('markets', []):
                if market['key'] == 'h2h':
                    for outcome in market['outcomes']:
                        if outcome['name'] == match['home_team']:
                            match_info['Home Win Odds'] = outcome['price']
                        elif outcome['name'] == match['away_team']:
                            match_info['Away Win Odds'] = outcome['price']
                        else:
                            match_info['Draw Odds'] = outcome['price']

        # Step 2: Fetch additional markets for the event
        event_response = requests.get(
            event_odds_url.format(eventId=match['id']),
            params={'apiKey': API_KEY, 'markets': 'btts,totals,correct_score', 'oddsFormat': 'decimal'}
        )

        matches_data.append(match_info)

    # Convert to DataFrame
    df = pd.DataFrame(matches_data)

    # Save results
    df.to_csv('premier_league_comprehensive_odds.csv', index=False)
    print("Data saved to 'premier_league_comprehensive_odds.csv'.")

else:
    print(f"Failed to retrieve data. Status Code: {response.status_code}")
    print("Error Message:", response.text)


Data saved to 'premier_league_comprehensive_odds.csv'.


In [187]:
import pandas as pd
import requests
import os

# Premier League sport key
API_KEY = 'ffb20d24ac64dccb79042ce2d0ab6e87'
SPORT_KEY = 'soccer_epl'

# Base API URLs
base_url = 'https://api.the-odds-api.com/v4/sports'
event_odds_url = 'https://api.the-odds-api.com/v4/events/{eventId}/odds'

# Parameters for initial odds fetch
params = {
    'apiKey': API_KEY,
    'regions': 'uk',
    'markets': 'h2h',  # Fetch basic h2h odds initially
    'oddsFormat': 'decimal'
}

# File to store historical data
history_file = 'premier_league_comprehensive_odds.csv'

# Step 1: Fetch matches
response = requests.get(f'{base_url}/{SPORT_KEY}/odds', params=params)

if response.status_code == 200:
    data = response.json()
    matches_data = []

    for match in data:
        match_info = {
            'Match ID': match['id'],
            'Date': match['commence_time'],
            'Home Team': match['home_team'],
            'Away Team': match['away_team']
        }

        # Extract h2h odds
        for bookmaker in match.get('bookmakers', []):
            for market in bookmaker.get('markets', []):
                if market['key'] == 'h2h':
                    for outcome in market['outcomes']:
                        if outcome['name'] == match['home_team']:
                            match_info['Home Win Odds'] = outcome['price']
                        elif outcome['name'] == match['away_team']:
                            match_info['Away Win Odds'] = outcome['price']
                        else:
                            match_info['Draw Odds'] = outcome['price']

        # Step 2: Fetch additional markets for the event
        event_response = requests.get(
            event_odds_url.format(eventId=match['id']),
            params={'apiKey': API_KEY, 'markets': 'btts,totals,correct_score', 'oddsFormat': 'decimal'}
        )

        matches_data.append(match_info)

    # Convert to DataFrame
    new_data_df = pd.DataFrame(matches_data)

    # Step 3: Check if history exists and merge with new data
    if os.path.exists(history_file):
        # Load historical data
        historical_df = pd.read_csv(history_file)

        # Merge new data with historical data (preserve old data where new data is missing)
        merged_df = pd.concat([historical_df, new_data_df]).drop_duplicates(subset=['Match ID'], keep='last')
    else:
        # If no historical data exists, use the new data as the initial dataset
        merged_df = new_data_df

    # Save updated dataset to CSV
    merged_df.to_csv(history_file, index=False)
    print(f"Data saved and updated to '{history_file}'.")

else:
    print(f"Failed to retrieve data. Status Code: {response.status_code}")
    print("Error Message:", response.text)


Data saved and updated to 'premier_league_comprehensive_odds.csv'.


In [185]:
print(match)

{'id': '86df5e4f56b948cd24fab60e998a43fc', 'sport_key': 'soccer_epl', 'sport_title': 'EPL', 'commence_time': '2024-12-09T20:00:00Z', 'home_team': 'West Ham United', 'away_team': 'Wolverhampton Wanderers', 'bookmakers': [{'key': 'smarkets', 'title': 'Smarkets', 'last_update': '2024-12-02T01:33:41Z', 'markets': [{'key': 'h2h', 'last_update': '2024-12-02T01:33:41Z', 'outcomes': [{'name': 'West Ham United', 'price': 1.96}, {'name': 'Wolverhampton Wanderers', 'price': 3.5}, {'name': 'Draw', 'price': 3.7}]}, {'key': 'h2h_lay', 'last_update': '2024-12-02T01:33:41Z', 'outcomes': [{'name': 'West Ham United', 'price': 2.0}, {'name': 'Wolverhampton Wanderers', 'price': 4.1}, {'name': 'Draw', 'price': 4.3}]}]}, {'key': 'paddypower', 'title': 'Paddy Power', 'last_update': '2024-12-02T01:32:58Z', 'markets': [{'key': 'h2h', 'last_update': '2024-12-02T01:32:58Z', 'outcomes': [{'name': 'West Ham United', 'price': 1.95}, {'name': 'Wolverhampton Wanderers', 'price': 3.75}, {'name': 'Draw', 'price': 3.6}]

In [6]:
df.columns

Index(['Match ID', 'Date', 'Home Team', 'Away Team', 'Away Win Odds',
       'Home Win Odds', 'Draw Odds'],
      dtype='object')

In [179]:
df.head(5)

,Match ID,Date,Home Team,Away Team,Away Win Odds,Home Win Odds,Draw Odds
0,1f3a141af7b6cda2db7c51cd49b69430,2024-12-03T19:30:00Z,Ipswich Town,Crystal Palace,2.40,2.63,2.7
1,882cefd62d702d18942a27b8bdabe217,2024-12-03T20:15:00Z,Leicester City,West Ham United,2.30,2.50,2.8
2,35a970f1185447b6ecb34579e3a9081f,2024-12-04T19:30:00Z,Southampton,Chelsea,1.33,6.50,4.2
3,cf50e11019bafbf2b5f32da174a2a4f5,2024-12-04T19:30:00Z,Everton,Wolverhampton Wanderers,3.25,2.00,2.8
4,d8a41f415b6d80b1639e4d7d3c9fd0cd,2024-12-04T19:30:00Z,Newcastle United,Liverpool,1.67,3.75,3.1


In [69]:
import requests

# FootyStats API Key and League ID
API_KEY = "test85g57"
LEAGUE_ID = 1625  # Replace with a valid league ID

# API URL
url = f"https://api.footystats.org/league-matches?key={API_KEY}&league_id={LEAGUE_ID}"
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    matches = data.get("data", [])
    
    # Print one match's details to inspect available fields
    if matches:
        print("Available fields in match data:")
        print(matches[0])  # Print the first match
    else:
        print("No matches found in the response.")
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")
    print(response.text)


Available fields in match data:
{'id': 453873, 'homeID': 149, 'awayID': 108, 'season': '2018/2019', 'status': 'complete', 'roundID': 3547, 'game_week': 1, 'revised_game_week': -1, 'homeGoals': ['3', '83'], 'awayGoals': ['90+2'], 'homeGoalCount': 2, 'awayGoalCount': 1, 'totalGoalCount': 3, 'team_a_corners': 2, 'team_b_corners': 5, 'totalCornerCount': 7, 'team_a_offsides': 4, 'team_b_offsides': 2, 'team_a_yellow_cards': 2, 'team_b_yellow_cards': 1, 'team_a_red_cards': 0, 'team_b_red_cards': 0, 'team_a_shotsOnTarget': 7, 'team_b_shotsOnTarget': 5, 'team_a_shotsOffTarget': 1, 'team_b_shotsOffTarget': 3, 'team_a_shots': 8, 'team_b_shots': 8, 'team_a_fouls': 11, 'team_b_fouls': 8, 'team_a_possession': 46, 'team_b_possession': 54, 'refereeID': 526, 'coach_a_ID': 208, 'coach_b_ID': 204, 'stadium_name': 'Old Trafford (Manchester)', 'stadium_location': '', 'team_a_cards_num': 2, 'team_b_cards_num': 1, 'odds_ft_1': 1.37, 'odds_ft_x': 4.98, 'odds_ft_2': 9.81, 'odds_ft_over05': 1.05, 'odds_ft_over1

In [197]:
odds_data_df = pd.read_csv("E0_1.csv")

In [201]:
odds_data_df.tail()

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,BFECAHH,BFECAHA
125,E0,30/11/2024,17:30,West Ham,Arsenal,2,5,A,2,5,...,1.92,1.98,1.94,1.98,1.95,2.02,1.90,1.96,1.98,2.01
126,E0,01/12/2024,13:30,Chelsea,Aston Villa,3,0,H,2,0,...,1.97,1.96,1.96,1.96,1.99,2.03,1.90,1.96,1.93,2.04
127,E0,01/12/2024,13:30,Man United,Everton,4,0,H,2,0,...,2.05,1.88,2.07,1.86,2.17,1.90,2.05,1.83,2.06,1.92
128,E0,01/12/2024,13:30,Tottenham,Fulham,1,1,D,0,0,...,2.00,1.93,1.99,1.93,2.00,1.96,1.95,1.91,1.99,2.00
129,E0,01/12/2024,16:00,Liverpool,Man City,2,0,H,1,0,...,2.07,1.86,2.07,1.86,2.08,1.89,2.03,1.84,2.10,1.89


In [206]:
import pandas as pd

# Define the standardized venue names and their coordinates
stadium_data = [
    {'Venue': 'Old Trafford', 'Latitude': 53.4631, 'Longitude': -2.2914},
    {'Venue': 'Portman Road Stadium', 'Latitude': 52.0567, 'Longitude': 1.1450},
    {'Venue': 'Emirates Stadium', 'Latitude': 51.5550, 'Longitude': -0.1086},
    {'Venue': 'Goodison Park', 'Latitude': 53.4389, 'Longitude': -2.9664},
    {'Venue': "St James' Park", 'Latitude': 54.9756, 'Longitude': -1.6217},
    {'Venue': 'The City Ground', 'Latitude': 52.9399, 'Longitude': -1.1325},
    {'Venue': 'London Stadium', 'Latitude': 51.5386, 'Longitude': -0.0164},
    {'Venue': 'Gtech Community Stadium', 'Latitude': 51.4908, 'Longitude': -0.2887},
    {'Venue': 'Stamford Bridge', 'Latitude': 51.4817, 'Longitude': -0.1911},
    {'Venue': 'King Power Stadium', 'Latitude': 52.6203, 'Longitude': -1.1422},
    {'Venue': 'The American Express Stadium', 'Latitude': 50.8618, 'Longitude': -0.0833},
    {'Venue': 'Selhurst Park', 'Latitude': 51.3983, 'Longitude': -0.0856},
    {'Venue': 'Craven Cottage', 'Latitude': 51.4750, 'Longitude': -0.2217},
    {'Venue': 'Etihad Stadium', 'Latitude': 53.4831, 'Longitude': -2.2003},
    {'Venue': "St Mary's Stadium", 'Latitude': 50.9058, 'Longitude': -1.3913},
    {'Venue': 'Tottenham Hotspur Stadium', 'Latitude': 51.6047, 'Longitude': -0.0664},
    {'Venue': 'Villa Park', 'Latitude': 52.5092, 'Longitude': -1.8840},
    {'Venue': 'Vitality Stadium', 'Latitude': 50.7353, 'Longitude': -1.8383},
    {'Venue': 'Molineux Stadium', 'Latitude': 52.5903, 'Longitude': -2.1303},
    {'Venue': 'Anfield', 'Latitude': 53.4308, 'Longitude': -2.9608}
]

# Create a DataFrame
stadium_df = pd.DataFrame(stadium_data)

# Add a primary key column
stadium_df.insert(0, 'StadiumID', range(1, len(stadium_df) + 1))

# Display the DataFrame
print(stadium_df)

# Optionally save to a CSV file
stadium_df.to_csv('premier_league_stadiums_location.csv', index=False)


    StadiumID                         Venue  Latitude  Longitude
0           1                  Old Trafford   53.4631    -2.2914
1           2          Portman Road Stadium   52.0567     1.1450
2           3              Emirates Stadium   51.5550    -0.1086
3           4                 Goodison Park   53.4389    -2.9664
4           5                St James' Park   54.9756    -1.6217
5           6               The City Ground   52.9399    -1.1325
6           7                London Stadium   51.5386    -0.0164
7           8       Gtech Community Stadium   51.4908    -0.2887
8           9               Stamford Bridge   51.4817    -0.1911
9          10            King Power Stadium   52.6203    -1.1422
10         11  The American Express Stadium   50.8618    -0.0833
11         12                 Selhurst Park   51.3983    -0.0856
12         13                Craven Cottage   51.4750    -0.2217
13         14                Etihad Stadium   53.4831    -2.2003
14         15            

In [210]:
## weather data directly in db management

## Stats data

In [224]:
API = 'cadf4921727a4fc98eb9e50304e95e1f'

In [232]:
import requests
import pandas as pd

# API key and base URL
API_KEY = API  # Replace with your actual API key
BASE_URL = 'https://api.football-data.org/v4'

# Headers for authentication
headers = {
    'X-Auth-Token': API_KEY
}

# Function to fetch match statistics for a specific match
def fetch_match_stats(match_id):
    # Match details endpoint
    url = f"{BASE_URL}/matches/{match_id}"
    
    # Make the API request
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        # Parse the JSON response
        match_data = response.json()
        
        # Extract relevant statistics
        stats = {
            "Match ID": match_id,
            "Date": match_data['utcDate'],
            "Home Team": match_data['homeTeam']['name'],
            "Away Team": match_data['awayTeam']['name'],
            "Shots Home": match_data['homeTeamStatistics']['shots'],
            "Shots Away": match_data['awayTeamStatistics']['shots'],
            "SOT Home": match_data['homeTeamStatistics']['shotsOnTarget'],
            "SOT Away": match_data['awayTeamStatistics']['shotsOnTarget'],
            "Possession Home": match_data['homeTeamStatistics']['possession'],
            "Possession Away": match_data['awayTeamStatistics']['possession'],
            "Corners Home": match_data['homeTeamStatistics']['corners'],
            "Corners Away": match_data['awayTeamStatistics']['corners'],
            "Cards Home": match_data['homeTeamStatistics']['cards'],
            "Cards Away": match_data['awayTeamStatistics']['cards'],
            "Offsides Home": match_data['homeTeamStatistics']['offsides'],
            "Offsides Away": match_data['awayTeamStatistics']['offsides']
        }
        
        return stats
    else:
        print(f"Failed to fetch data for match ID {match_id}. Status code: {response.status_code}")
        return None

# Example usage
match_ids = ['your_match_id_1', 'your_match_id_2']  # Replace with actual match IDs
match_stats_list = []

# Fetch stats for each match
for match_id in match_ids:
    stats = fetch_match_stats(match_id)
    if stats:
        match_stats_list.append(stats)

# Convert the list of stats to a DataFrame
df = pd.DataFrame(match_stats_list)

# Save to CSV
df.to_csv('match_statistics.csv', index=False)
print("Match statistics saved to 'match_statistics.csv'.")


Failed to fetch data for match ID your_match_id_1. Status code: 400
Failed to fetch data for match ID your_match_id_2. Status code: 400
Match statistics saved to 'match_statistics.csv'.


In [234]:
# Fetch all matches in the Premier League
url = f"{BASE_URL}/competitions/PL/matches"
response = requests.get(url, headers=headers)

if response.status_code == 200:
    matches = response.json()
    match_ids = [match['id'] for match in matches['matches']]
    print("Valid match IDs:", match_ids)
else:
    print(f"Failed to fetch match list. Status code: {response.status_code}")
    print("Error message:", response.text)


Valid match IDs: [497410, 497411, 497412, 497413, 497414, 497415, 497416, 497417, 497418, 497419, 497422, 497423, 497424, 497426, 497427, 497428, 497421, 497420, 497429, 497425, 497430, 497431, 497433, 497434, 497435, 497438, 497439, 497432, 497437, 497436, 497447, 497442, 497443, 497444, 497445, 497446, 497441, 497440, 497448, 497449, 497459, 497450, 497453, 497454, 497455, 497457, 497458, 497452, 497451, 497456, 497467, 497461, 497462, 497463, 497464, 497468, 497469, 497465, 497466, 497460, 497475, 497470, 497472, 497477, 497478, 497479, 497476, 497471, 497474, 497473, 497488, 497481, 497482, 497484, 497485, 497487, 497480, 497489, 497483, 497486, 497497, 497491, 497492, 497493, 497498, 497496, 497494, 497495, 497499, 497490, 497505, 497500, 497502, 497503, 497506, 497507, 497509, 497508, 497504, 497501, 497510, 497513, 497518, 497519, 497511, 497514, 497515, 497516, 497517, 497512, 497526, 497520, 497521, 497522, 497523, 497524, 497527, 497529, 497525, 497528, 497531, 497530, 497533

In [269]:

# Function to fetch match statistics
def fetch_match_stats(match_id):
    url = f"{BASE_URL}/matches/{match_id}"
    headers = {'X-Auth-Token': API_KEY}
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        match_data = response.json()
        # Print the raw JSON response to verify structure
        print(f"Match Data for ID {match_id}:\n", match_data)
        
        # Check if match is completed
        if match_data.get('status') == 'FINISHED':
            stats = {
                "Match ID": match_id,
                "Date": match_data['utcDate'],
                "Home Team": match_data['homeTeam']['name'],
                "Away Team": match_data['awayTeam']['name'],
                # Fixed score keys to match the API response structure
                "Score": f"{match_data['score']['fullTime']['home']} - {match_data['score']['fullTime']['away']}",
                # Statistics like shots and possession are not in this response
                # Placeholder values added to avoid KeyError
                "Shots Home": match_data.get('homeTeamStatistics', {}).get('shots', None),
                "Shots Away": match_data.get('awayTeamStatistics', {}).get('shots', None),
                "Possession Home": match_data.get('homeTeamStatistics', {}).get('possession', None),
                "Possession Away": match_data.get('awayTeamStatistics', {}).get('possession', None),
                "Corners Home": match_data.get('homeTeamStatistics', {}).get('corners', None),
                "Corners Away": match_data.get('awayTeamStatistics', {}).get('corners', None),
            }
            print("Match Statistics:\n", stats)
        else:
            print(f"Match ID {match_id} is not completed yet.")
    else:
        print(f"Failed to fetch data for match ID {match_id}. Status code: {response.status_code}")
        if response.status_code == 429:
            print("Rate limit exceeded. Wait and retry.")


# Test with Match ID: 497410
fetch_match_stats(497410)


Match Data for ID 497410:
 {'area': {'id': 2072, 'name': 'England', 'code': 'ENG', 'flag': 'https://crests.football-data.org/770.svg'}, 'competition': {'id': 2021, 'name': 'Premier League', 'code': 'PL', 'type': 'LEAGUE', 'emblem': 'https://crests.football-data.org/PL.png'}, 'season': {'id': 2287, 'startDate': '2024-08-16', 'endDate': '2025-05-25', 'currentMatchday': 14, 'winner': None}, 'id': 497410, 'utcDate': '2024-08-16T19:00:00Z', 'status': 'FINISHED', 'venue': None, 'matchday': 1, 'stage': 'REGULAR_SEASON', 'group': None, 'lastUpdated': '2024-12-03T00:20:55Z', 'homeTeam': {'id': 66, 'name': 'Manchester United FC', 'shortName': 'Man United', 'tla': 'MUN', 'crest': 'https://crests.football-data.org/66.png'}, 'awayTeam': {'id': 63, 'name': 'Fulham FC', 'shortName': 'Fulham', 'tla': 'FUL', 'crest': 'https://crests.football-data.org/63.png'}, 'score': {'winner': 'HOME_TEAM', 'duration': 'REGULAR', 'fullTime': {'home': 1, 'away': 0}, 'halfTime': {'home': 0, 'away': 0}}, 'odds': {'msg':

## Head to Head data

In [9]:
import requests
import pandas as pd
from itertools import combinations

# API key and endpoint
API_KEY = 'bff3a4a60fa8c8b4c7ba3b6a7a7f3365'  # Replace with your actual API key
BASE_URL = 'https://v3.football.api-sports.io/fixtures/headtohead'

# Headers for authentication
headers = {
    'x-rapidapi-host': 'v3.football.api-sports.io',
    'x-rapidapi-key': API_KEY
}

# Function to fetch and process H2H data
def fetch_and_process_h2h_data(home_team_id, away_team_id):
    params = {'h2h': f'{home_team_id}-{away_team_id}'}
    response = requests.get(BASE_URL, headers=headers, params=params)
    
    if response.status_code == 200:
        data = response.json()
        matches = []
        
        # Extract the relevant information
        for fixture in data.get('response', []):
            match = fixture.get('fixture', {})
            teams = fixture.get('teams', {})
            score = fixture.get('score', {}).get('fulltime', {})
            
            matches.append({
                "Date": match.get('date'),
                "HT": teams.get('home', {}).get('name'),
                "AT": teams.get('away', {}).get('name'),
                "HT_Score": score.get('home'),
                "AT_Score": score.get('away')
            })
        
        # Convert to DataFrame
        return pd.DataFrame(matches)
    else:
        print(f"Failed to retrieve data for teams {home_team_id} vs {away_team_id}. Status code: {response.status_code}")
        print("Response:", response.text)
        return pd.DataFrame()

# Main logic to process all team pairs
def fetch_all_h2h_data(team_ids):
    all_h2h_data = []

    # Iterate through all team pairs
    for home_team_id, away_team_id in combinations(team_ids, 2):
        print(f"Processing H2H data for teams {home_team_id} and {away_team_id}")
        h2h_data = fetch_and_process_h2h_data(home_team_id, away_team_id)
        if not h2h_data.empty:
            all_h2h_data.append(h2h_data)

    # Combine all data into a single DataFrame
    if all_h2h_data:  # Ensure there's data to concatenate
        final_h2h_table = pd.concat(all_h2h_data, ignore_index=True)
        final_h2h_table['Date'] = pd.to_datetime(final_h2h_table['Date'])
        final_h2h_table = final_h2h_table.sort_values(by='Date').reset_index(drop=True)
        return final_h2h_table
    else:
        print("No H2H data available for the specified teams.")
        return pd.DataFrame()

# Fetch and process H2H data for all team pairs
team_ids = range(33, 53)  # Team IDs from 33 to 52
final_h2h_table = fetch_all_h2h_data(team_ids)

# Save the final table to a CSV file
if not final_h2h_table.empty:
    final_h2h_table.to_csv("all_h2h_fixtures.csv", index=False)
    print("All H2H data saved to 'all_h2h_fixtures.csv'.")
else:
    print("No H2H data to save.")


Processing H2H data for teams 33 and 34
Processing H2H data for teams 33 and 35
Processing H2H data for teams 33 and 36
Processing H2H data for teams 33 and 37
Processing H2H data for teams 33 and 38
Processing H2H data for teams 33 and 39
Processing H2H data for teams 33 and 40
Processing H2H data for teams 33 and 41
Processing H2H data for teams 33 and 42
Processing H2H data for teams 33 and 43
Processing H2H data for teams 33 and 44
Processing H2H data for teams 33 and 45
Processing H2H data for teams 33 and 46
Processing H2H data for teams 33 and 47
Processing H2H data for teams 33 and 48
Processing H2H data for teams 33 and 49
Processing H2H data for teams 33 and 50
Processing H2H data for teams 33 and 51
Processing H2H data for teams 33 and 52
Processing H2H data for teams 34 and 35
Processing H2H data for teams 34 and 36
Processing H2H data for teams 34 and 37
Processing H2H data for teams 34 and 38
Processing H2H data for teams 34 and 39
Processing H2H data for teams 34 and 40


In [11]:
data = final_h2h_table
data.head()

,Date,HT,AT,HT_Score,AT_Score
0,2010-08-16 19:00:00+00:00,Manchester United,Newcastle,3.0,0.0
1,2010-08-22 15:00:00+00:00,Fulham,Manchester United,2.0,2.0
2,2010-09-19 12:30:00+00:00,Manchester United,Liverpool,3.0,2.0
3,2010-10-23 11:45:00+00:00,Tottenham,Everton,1.0,1.0
4,2010-11-06 15:00:00+00:00,Manchester United,Wolves,2.0,1.0


In [185]:
len(data)

425

In [23]:
import pandas as pd

# Convert the 'Date' column to datetime
data['Date'] = pd.to_datetime(data['Date'])

# Separate date and time into two columns
data['Date_Only'] = data['Date'].dt.date
data['Time'] = data['Date'].dt.time

# Remove rows where the date is less than 2022
filtered_data = data[data['Date_Only'] >= pd.to_datetime('2021-01-01').date()]

# Drop the original 'Date' column if no longer needed
filtered_data = filtered_data.drop(columns=['Date'])

#missing values removal
filtered_data = filtered_data.dropna()

# Save the filtered data to a new CSV file
filtered_data.to_csv('filtered_h2h_data.csv', index=False)

# Rename columns
filtered_data.rename(columns={'Date_Only': 'Date'}, inplace=True)


print("Filtered data has been saved to 'filtered_h2h_data.csv'.")

Filtered data has been saved to 'filtered_h2h_data.csv'.


In [25]:
len(filtered_data)

140

## SoccerData

In [42]:
#python3 -m pip install soccerdata
# pip install socceraction

In [8]:
import soccerdata as sd


[12/07/24 19:41:36] INFO     No custom team name replacements found. You can configure these in       ]8;id=933241;file:///opt/anaconda3/lib/python3.11/site-packages/soccerdata/_config.py\_config.py]8;;\:]8;id=127263;file:///opt/anaconda3/lib/python3.11/site-packages/soccerdata/_config.py#90\90]8;;\
                             /Users/manish/soccerdata/config/teamname_replacements.json.                           

                    INFO     No custom league dict found. You can configure additional leagues in    ]8;id=915959;file:///opt/anaconda3/lib/python3.11/site-packages/soccerdata/_config.py\_config.py]8;;\:]8;id=894717;file:///opt/anaconda3/lib/python3.11/site-packages/soccerdata/_config.py#196\196]8;;\
                             /Users/manish/soccerdata/config/league_dict.json.                                     

In [18]:
understat = sd.Understat(leagues="ENG-Premier League", seasons="2024/2025")
print(understat.__doc__)

[12/07/24 19:48:21] INFO     Saving cached data to /Users/manish/soccerdata/data/Understat           ]8;id=577659;file:///opt/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=802840;file:///opt/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py#263\263]8;;\

Provides pd.DataFrames from data at https://understat.com.

    Data will be downloaded as necessary and cached locally in
    ``~/soccerdata/data/Understat``.

    Parameters
    ----------
    proxy : 'tor' or dict or list(dict) or callable, optional
        Use a proxy to hide your IP address. Valid options are:
            - "tor": Uses the Tor network. Tor should be running in
              the background on port 9050.
            - dict: A dictionary with the proxy to use. The dict should be
              a mapping of supported protocols to proxy addresses. For example::

                  {
                      'http': 'http://10.10.1.10:3128',
                      'https': 'http://10.10.1.10:1080',
                  }

            - list(dict): A list of proxies to choose from. A different proxy will
              be selected from this list after failed requests, allowing rotating
              proxies.
            - callable: A function that returns a valid proxy. This funct

In [20]:
leagues = understat.read_leagues()
leagues.head()

,league_id,url
league,,
ENG-Premier League,1,https://understat.com/league/EPL


In [22]:
schedule = understat.read_schedule()
schedule.head()

league_id  \
league             season game                                                   
ENG-Premier League 2425   2024-08-16 Manchester United-Fulham                1   
                          2024-08-17 Arsenal-Wolverhampton Wanderers         1   
                          2024-08-17 Everton-Brighton                        1   
                          2024-08-17 Ipswich-Liverpool                       1   
                          2024-08-17 Newcastle United-Southampton            1   

                                                                      season_id  \
league             season game                                                    
ENG-Premier League 2425   2024-08-16 Manchester United-Fulham              2024   
                          2024-08-17 Arsenal-Wolverhampton Wanderers       2024   
                          2024-08-17 Everton-Brighton                      2024   
                          2024-08-17 Ipswich-Liverpool                     2024   
                          2024-08-17 Newcastle United-Southampton          2024   

                                                                      game_id  \
league             season game                                                  
ENG-Premier League 2425   2024-08-16 Manchester United-Fulham           26602   
                          2024-08-17 Arsenal-Wolverhampton Wanderers    26604   
                          2024-08-17 Everton-Brighton                   26605   
                          2024-08-17 Ipswich-Liverpool                  26603   
                          2024-08-17 Newcastle United-Southampton       26606   

                                                                                    date  \
league             season game                                                             
ENG-Premier League 2425   2024-08-16 Manchester United-Fulham        2024-08-16 19:00:00   
                          2024-08-17 Arsenal-Wolverhampton Wanderers 2024-08-17 14:00:00   
                          2024-08-17 Everton-Brighton                2024-08-17 14:00:00   
                          2024-08-17 Ipswich-Liverpool               2024-08-17 11:30:00   
                          2024-08-17 Newcastle United-Southampton    2024-08-17 14:00:00   

                                                                      home_team_id  \
league             season game                                                       
ENG-Premier League 2425   2024-08-16 Manchester United-Fulham                   89   
                          2024-08-17 Arsenal-Wolverhampton Wanderers            83   
                          2024-08-17 Everton-Brighton                           72   
                          2024-08-17 Ipswich-Liverpool                         285   
                          2024-08-17 Newcastle United-Southampton               86   

                                                                      away_team_id  \
league             season game                                                       
ENG-Premier League 2425   2024-08-16 Manchester United-Fulham                  228   
                          2024-08-17 Arsenal-Wolverhampton Wanderers           229   
                          2024-08-17 Everton-Brighton                          220   
                          2024-08-17 Ipswich-Liverpool                          87   
                          2024-08-17 Newcastle United-Southampton               74   

                                                                              home_team  \
league             season game                                                            
ENG-Premier League 2425   2024-08-16 Manchester United-Fulham         Manchester United   
                          2024-08-17 Arsenal-Wolverhampton Wanderers            Arsenal   
                          2024-08-17 Everton-Brighton                           Everton   
                          2024-08-17 I

In [113]:
schedule.to_csv("schedule.csv")

In [117]:
team_match_stats = understat.read_team_match_stats()
team_match_stats.tail()


league_id  \
league             season game                                                       
ENG-Premier League 2425   2024-12-07 Manchester United-Nottingham Forest         1   
                          2024-12-08 Fulham-Arsenal                              1   
                          2024-12-08 Ipswich-Bournemouth                         1   
                          2024-12-08 Leicester-Brighton                          1   
                          2024-12-08 Tottenham-Chelsea                           1   

                                                                          season_id  \
league             season game                                                        
ENG-Premier League 2425   2024-12-07 Manchester United-Nottingham Forest       2024   
                          2024-12-08 Fulham-Arsenal                            2024   
                          2024-12-08 Ipswich-Bournemouth                       2024   
                          2024-12-08 Leicester-Brighton                        2024   
                          2024-12-08 Tottenham-Chelsea                         2024   

                                                                          game_id  \
league             season game                                                      
ENG-Premier League 2425   2024-12-07 Manchester United-Nottingham Forest    26744   
                          2024-12-08 Fulham-Arsenal                         26749   
                          2024-12-08 Ipswich-Bournemouth                    26750   
                          2024-12-08 Leicester-Brighton                     26751   
                          2024-12-08 Tottenham-Chelsea                      26745   

                                                                                        date  \
league             season game                                                                 
ENG-Premier League 2425   2024-12-07 Manchester United-Nottingham Forest 2024-12-07 17:30:00   
                          2024-12-08 Fulham-Arsenal                      2024-12-08 14:00:00   
                          2024-12-08 Ipswich-Bournemouth                 2024-12-08 14:00:00   
                          2024-12-08 Leicester-Brighton                  2024-12-08 14:00:00   
                          2024-12-08 Tottenham-Chelsea                   2024-12-08 16:30:00   

                                                                          home_team_id  \
league             season game                                                           
ENG-Premier League 2425   2024-12-07 Manchester United-Nottingham Forest            89   
                          2024-12-08 Fulham-Arsenal                                228   
                          2024-12-08 Ipswich-Bournemouth                           285   
                          2024-12-08 Leicester-Brighton                             75   
                          2024-12-08 Tottenham-Chelsea                              82   

                                                                          away_team_id  \
league             season game                                                           
ENG-Premier League 2425   2024-12-07 Manchester United-Nottingham Forest           249   
                          2024-12-08 Fulham-Arsenal                                 83   
                          2024-12-08 Ipswich-Bournemouth                            73   
                          2024-12-08 Leicester-Brighton                            220   
                          2024-12-08 Tottenham-Chelsea                              80   

                                                                                  home_team  \
league             season game                                                                
ENG-Premier League 2425   2024-12-07 Manchester United-Nottingham Forest  Manchester United   
                          2024-12-08 Fulham-

In [119]:
team_match_stats.to_csv("team_match_stats.csv")

In [30]:
player_season_stats = understat.read_player_season_stats()
player_season_stats.head()

league_id  season_id  team_id  \
league             season team    player                                        
ENG-Premier League 2425   Arsenal Ben White             1       2024       83   
                                  Bukayo Saka           1       2024       83   
                                  David Raya            1       2024       83   
                                  Declan Rice           1       2024       83   
                                  Ethan Nwaneri         1       2024       83   

                                                 player_id position  matches  \
league             season team    player                                       
ENG-Premier League 2425   Arsenal Ben White           7298      D S        9   
                                  Bukayo Saka         7322      F M       13   
                                  David Raya          9676       GK       14   
                                  Declan Rice         5553        M       12   
                                  Ethan Nwaneri      11132        S        7   

                                                 minutes  goals        xg  \
league             season team    player                                    
ENG-Premier League 2425   Arsenal Ben White          681      0  0.113899   
                                  Bukayo Saka       1107      5  4.644792   
                                  David Raya        1260      0       0.0   
                                  Declan Rice       1023      0  0.944568   
                                  Ethan Nwaneri       39      1  0.259517   

                                                 np_goals     np_xg  assists  \
league             season team    player                                       
ENG-Premier League 2425   Arsenal Ben White             0  0.113899        1   
                                  Bukayo Saka           4  3.883623       10   
                                  David Raya            0       0.0        0   
                                  Declan Rice           0  0.944568        2   
                                  Ethan Nwaneri         1  0.259517        0   

                                                       xa  shots  key_passes  \
league             season team    player                                       
ENG-Premier League 2425   Arsenal Ben White      0.411599      4           3   
                                  Bukayo Saka    8.708694     41          38   
                                  David Raya          0.0      0           0   
                                  Declan Rice    2.538483     12          20   
                                  Ethan Nwaneri  0.143631      4           2   

                                                 yellow_cards  red_cards  \
league             season team    player                                   
ENG-Premier League 2425   Arsenal Ben White                 2          0   
                                  Bukayo Saka               3          0   
                                  David Raya                2          0   
                                  Declan Rice               2          1   
                                  Ethan Nwaneri             0          0   

                                                 xg_chain  xg_buildup  
league             season team    player                               
ENG-Premier League 2425   Arsenal Ben White      2.050103    1.955298  
                                  Bukayo Saka    9.937323      3.9378  
                                  David Raya     2.090405    2.090405  
                                  Declan Rice    5.591398    3.601645  
                                  Ethan Nwaneri  1.042778    0.812131

In [99]:
mh = sd.MatchHistory(leagues="ENG-Premier League", seasons=2024)
print(mh.__doc__)

[12/07/24 20:50:49] INFO     Saving cached data to /Users/manish/soccerdata/data/MatchHistory        ]8;id=657324;file:///opt/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=373124;file:///opt/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py#263\263]8;;\

Provides pd.DataFrames from CSV files available at http://www.football-data.co.uk/data.php.

    Data will be downloaded as necessary and cached locally in
    ``~/soccerdata/data/MatchHistory``.

    Parameters
    ----------
    leagues : string or iterable
        IDs of leagues to include.
    seasons : string, int or list
        Seasons to include. Supports multiple formats.
        Examples: '16-17'; 2016; '2016-17'; [14, 15, 16]
    proxy : 'tor' or dict or list(dict) or callable, optional
        Use a proxy to hide your IP address. Valid options are:
            - "tor": Uses the Tor network. Tor should be running in
              the background on port 9050.
            - dict: A dictionary with the proxy to use. The dict should be
              a mapping of supported protocols to proxy addresses. For example::

                  {
                      'http': 'http://10.10.1.10:3128',
                      'https': 'http://10.10.1.10:1080',
                  }

           

In [103]:
# hist = mh.read_games()
# hist.head()


### Game schedule and predicted results

In [105]:
fbref = sd.FBref(leagues="ENG-Premier League", seasons=2024)
print(fbref.__doc__)

[12/07/24 20:55:06] INFO     Saving cached data to /Users/manish/soccerdata/data/FBref               ]8;id=641365;file:///opt/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=38795;file:///opt/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py#263\263]8;;\

Provides pd.DataFrames from data at http://fbref.com.

    Data will be downloaded as necessary and cached locally in
    ``~/soccerdata/data/FBref``.

    Parameters
    ----------
    leagues : string or iterable, optional
        IDs of leagues to include. For efficiently reading data from the Top-5
        European leagues, use "Big 5 European Leagues Combined".
    seasons : string, int or list, optional
        Seasons to include. Supports multiple formats.
        Examples: '16-17'; 2016; '2016-17'; [14, 15, 16]
    proxy : 'tor' or dict or list(dict) or callable, optional
        Use a proxy to hide your IP address. Valid options are:
            - "tor": Uses the Tor network. Tor should be running in
              the background on port 9050.
            - dict: A dictionary with the proxy to use. The dict should be
              a mapping of supported protocols to proxy addresses. For example::

                  {
                      'http': 'http://10.10.1.10:3128',
     

In [123]:
team_match_stats = fbref.read_team_match_stats(stat_type="goal_shot_creation")
team_match_stats.head()

/opt/anaconda3/lib/python3.11/site-packages/soccerdata/fbref.py:159: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pd.concat(dfs)


date  \
                                                                                  
league             season team    game                                            
ENG-Premier League 2425   Arsenal 2024-08-17 Arsenal-Wolves          2024-08-17   
                                  2024-08-24 Aston Villa-Arsenal     2024-08-24   
                                  2024-08-31 Arsenal-Brighton        2024-08-31   
                                  2024-09-15 Tottenham-Arsenal       2024-09-15   
                                  2024-09-22 Manchester City-Arsenal 2024-09-22   

                                                                            round  \
                                                                                    
league             season team    game                                              
ENG-Premier League 2425   Arsenal 2024-08-17 Arsenal-Wolves           Matchweek 1   
                                  2024-08-24 Aston Villa-Arsenal      Matchweek 2   
                                  2024-08-31 Arsenal-Brighton         Matchweek 3   
                                  2024-09-15 Tottenham-Arsenal        Matchweek 4   
                                  2024-09-22 Manchester City-Arsenal  Matchweek 5   

                                                                      day  \
                                                                            
league             season team    game                                      
ENG-Premier League 2425   Arsenal 2024-08-17 Arsenal-Wolves           Sat   
                                  2024-08-24 Aston Villa-Arsenal      Sat   
                                  2024-08-31 Arsenal-Brighton         Sat   
                                  2024-09-15 Tottenham-Arsenal        Sun   
                                  2024-09-22 Manchester City-Arsenal  Sun   

                                                                     venue  \
                                                                             
league             season team    game                                       
ENG-Premier League 2425   Arsenal 2024-08-17 Arsenal-Wolves           Home   
                                  2024-08-24 Aston Villa-Arsenal      Away   
                                  2024-08-31 Arsenal-Brighton         Home   
                                  2024-09-15 Tottenham-Arsenal        Away   
                                  2024-09-22 Manchester City-Arsenal  Away   

                                                                     result  \
                                                                              
league             season team    game                                        
ENG-Premier League 2425   Arsenal 2024-08-17 Arsenal-Wolves               W   
                                  2024-08-24 Aston Villa-Arsenal          W   
                                  2024-08-31 Arsenal-Brighton             D   
                                  2024-09-15 Tottenham-Arsenal            W   
                                  2024-09-22 Manchester City-Arsenal      D   

                                                                     GF GA  \
                                                                             
league             season team    game                                       
ENG-Premier League 2425   Arsenal 2024-08-17 Arsenal-Wolves           2  0   
                                  2024-08-24 Aston Villa-Arsenal      2  0   
                                  2024-08-31 Arsenal-Brighton         1  1   
                                  2024-09-15 Tottenham-Arsenal        1  0   
                                  2024-09-22 Manchester City-Arsenal  2  2   

                                                                             opponent  \
                                                                                        
league             season team    game             

In [127]:
team_match_stats = fbref.read_team_match_stats(stat_type="misc")
team_match_stats.head()

/opt/anaconda3/lib/python3.11/site-packages/soccerdata/fbref.py:159: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pd.concat(dfs)


date  \
                                                                                  
league             season team    game                                            
ENG-Premier League 2425   Arsenal 2024-08-17 Arsenal-Wolves          2024-08-17   
                                  2024-08-24 Aston Villa-Arsenal     2024-08-24   
                                  2024-08-31 Arsenal-Brighton        2024-08-31   
                                  2024-09-15 Tottenham-Arsenal       2024-09-15   
                                  2024-09-22 Manchester City-Arsenal 2024-09-22   

                                                                            round  \
                                                                                    
league             season team    game                                              
ENG-Premier League 2425   Arsenal 2024-08-17 Arsenal-Wolves           Matchweek 1   
                                  2024-08-24 Aston Villa-Arsenal      Matchweek 2   
                                  2024-08-31 Arsenal-Brighton         Matchweek 3   
                                  2024-09-15 Tottenham-Arsenal        Matchweek 4   
                                  2024-09-22 Manchester City-Arsenal  Matchweek 5   

                                                                      day  \
                                                                            
league             season team    game                                      
ENG-Premier League 2425   Arsenal 2024-08-17 Arsenal-Wolves           Sat   
                                  2024-08-24 Aston Villa-Arsenal      Sat   
                                  2024-08-31 Arsenal-Brighton         Sat   
                                  2024-09-15 Tottenham-Arsenal        Sun   
                                  2024-09-22 Manchester City-Arsenal  Sun   

                                                                     venue  \
                                                                             
league             season team    game                                       
ENG-Premier League 2425   Arsenal 2024-08-17 Arsenal-Wolves           Home   
                                  2024-08-24 Aston Villa-Arsenal      Away   
                                  2024-08-31 Arsenal-Brighton         Home   
                                  2024-09-15 Tottenham-Arsenal        Away   
                                  2024-09-22 Manchester City-Arsenal  Away   

                                                                     result  \
                                                                              
league             season team    game                                        
ENG-Premier League 2425   Arsenal 2024-08-17 Arsenal-Wolves               W   
                                  2024-08-24 Aston Villa-Arsenal          W   
                                  2024-08-31 Arsenal-Brighton             D   
                                  2024-09-15 Tottenham-Arsenal            W   
                                  2024-09-22 Manchester City-Arsenal      D   

                                                                     GF GA  \
                                                                             
league             season team    game                                       
ENG-Premier League 2425   Arsenal 2024-08-17 Arsenal-Wolves           2  0   
                                  2024-08-24 Aston Villa-Arsenal      2  0   
                                  2024-08-31 Arsenal-Brighton         1  1   
                                  2024-09-15 Tottenham-Arsenal        1  0   
                                  2024-09-22 Manchester City-Arsenal  2  2   

                                                                             opponent  \
                                                                                        
league             season team    game             

In [129]:

# stat_type should be in ['schedule', 'shooting', 'keeper', 'passing', 'passing_types', 'goal_shot_creation', 'defense', 'possession', 'misc']
team_match_stats = fbref.read_team_match_stats(stat_type="defense")
team_match_stats.head()

/opt/anaconda3/lib/python3.11/site-packages/soccerdata/fbref.py:159: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pd.concat(dfs)


date  \
                                                                                  
league             season team    game                                            
ENG-Premier League 2425   Arsenal 2024-08-17 Arsenal-Wolves          2024-08-17   
                                  2024-08-24 Aston Villa-Arsenal     2024-08-24   
                                  2024-08-31 Arsenal-Brighton        2024-08-31   
                                  2024-09-15 Tottenham-Arsenal       2024-09-15   
                                  2024-09-22 Manchester City-Arsenal 2024-09-22   

                                                                            round  \
                                                                                    
league             season team    game                                              
ENG-Premier League 2425   Arsenal 2024-08-17 Arsenal-Wolves           Matchweek 1   
                                  2024-08-24 Aston Villa-Arsenal      Matchweek 2   
                                  2024-08-31 Arsenal-Brighton         Matchweek 3   
                                  2024-09-15 Tottenham-Arsenal        Matchweek 4   
                                  2024-09-22 Manchester City-Arsenal  Matchweek 5   

                                                                      day  \
                                                                            
league             season team    game                                      
ENG-Premier League 2425   Arsenal 2024-08-17 Arsenal-Wolves           Sat   
                                  2024-08-24 Aston Villa-Arsenal      Sat   
                                  2024-08-31 Arsenal-Brighton         Sat   
                                  2024-09-15 Tottenham-Arsenal        Sun   
                                  2024-09-22 Manchester City-Arsenal  Sun   

                                                                     venue  \
                                                                             
league             season team    game                                       
ENG-Premier League 2425   Arsenal 2024-08-17 Arsenal-Wolves           Home   
                                  2024-08-24 Aston Villa-Arsenal      Away   
                                  2024-08-31 Arsenal-Brighton         Home   
                                  2024-09-15 Tottenham-Arsenal        Away   
                                  2024-09-22 Manchester City-Arsenal  Away   

                                                                     result  \
                                                                              
league             season team    game                                        
ENG-Premier League 2425   Arsenal 2024-08-17 Arsenal-Wolves               W   
                                  2024-08-24 Aston Villa-Arsenal          W   
                                  2024-08-31 Arsenal-Brighton             D   
                                  2024-09-15 Tottenham-Arsenal            W   
                                  2024-09-22 Manchester City-Arsenal      D   

                                                                     GF GA  \
                                                                             
league             season team    game                                       
ENG-Premier League 2425   Arsenal 2024-08-17 Arsenal-Wolves           2  0   
                                  2024-08-24 Aston Villa-Arsenal      2  0   
                                  2024-08-31 Arsenal-Brighton         1  1   
                                  2024-09-15 Tottenham-Arsenal        1  0   
                                  2024-09-22 Manchester City-Arsenal  2  2   

                                                                             opponent  \
                                                                                        
league             season team    game             

In [133]:
player_season_stats = fbref.read_player_season_stats(stat_type="standard")
player_season_stats.head()

/opt/anaconda3/lib/python3.11/site-packages/soccerdata/fbref.py:159: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pd.concat(dfs)


nation    pos     age  born  \
                                                                              
league             season team    player                                      
ENG-Premier League 2425   Arsenal Ben White        ENG     DF  27-062  1997   
                                  Bukayo Saka      ENG  FW,MF  23-095  2001   
                                  David Raya       ESP     GK  29-085  1995   
                                  Declan Rice      ENG     MF  25-330  1999   
                                  Ethan Nwaneri    ENG  MF,FW  17-263  2007   

                                                Playing Time               \
                                                          MP Starts   Min   
league             season team    player                                    
ENG-Premier League 2425   Arsenal Ben White                9      7   695   
                                  Bukayo Saka             14     14  1155   
                                  David Raya              15     15  1350   
                                  Declan Rice             13     13  1102   
                                  Ethan Nwaneri            8      0    83   

                                                      Performance          \
                                                  90s         Gls Ast G+A   
league             season team    player                                    
ENG-Premier League 2425   Arsenal Ben White       7.7           0   1   1   
                                  Bukayo Saka    12.8           5  10  15   
                                  David Raya     15.0           0   0   0   
                                  Declan Rice    12.2           0   2   2   
                                  Ethan Nwaneri   0.9           1   0   1   

                                                                         \
                                                G-PK PK PKatt CrdY CrdR   
league             season team    player                                  
ENG-Premier League 2425   Arsenal Ben White        0  0     0    2    0   
                                  Bukayo Saka      4  1     1    3    0   
                                  David Raya       0  0     0    2    0   
                                  Declan Rice      0  0     0    5    1   
                                  Ethan Nwaneri    1  0     0    0    0   

                                                Expected                     \
                                                      xG npxG  xAG npxG+xAG   
league             season team    player                                      
ENG-Premier League 2425   Arsenal Ben White          0.1  0.1  0.4      0.6   
                                  Bukayo Saka        3.9  3.1  5.5      8.5   
                                  David Raya         0.0  0.0  0.0      0.0   
                                  Declan Rice        0.9  0.9  2.7      3.6   
                                  Ethan Nwaneri      0.2  0.2  0.2      0.3   

                                                Progression            \
                                                       PrgC PrgP PrgR   
league             season team    player                                
ENG-Premier League 2425   Arsenal Ben White               4   30   15   
                                  Bukayo Saka            65   47  173   
                                  David Raya              0    3    0   
                                  Declan Rice            35   79   39   
                                  Ethan Nwaneri           6    5    7   

                                                Per 90 Minutes              \
                                                           Gls   Ast   G+A   
league             season team    player                                     
ENG-Premier League 2425   Arsenal Ben White                0.0  0.13  0.13   
                                  Bukayo Saka     

In [153]:
# Function to fetch referee data for a match
def fetch_referee_data(match_id):
    url = f"{base_url}{match_id}"
    headers = {'X-Auth-Token': API_KEY}
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        match_data = response.json()
        try:
            # Extract the main referee's name
            referee = match_data['referees'][0]['name']
            return referee
        except (KeyError, IndexError):
            return "Unknown"
    else:
        print(f"Failed to fetch data for Match ID {match_id}: {response.status_code}")
        return "Unknown"

# Example Usage
example_match_id = 497410
referee = fetch_referee_data(example_match_id)
print(f"Referee for Match {example_match_id}: {referee}")


Referee for Match 497410: Robert Jones


In [192]:
import pandas as pd
import requests
import time

# Your API Key
API_KEY = 'cadf4921727a4fc98eb9e50304e95e1f'

# API Endpoint
base_url = "https://api.football-data.org/v4/matches/"

# Function to fetch referee data for a match
def fetch_referee_data(match_id):
    url = f"{base_url}{match_id}"
    headers = {'X-Auth-Token': API_KEY}
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            match_data = response.json()
            # Extract the main referee's name if available
            if 'referees' in match_data and len(match_data['referees']) > 0:
                return match_data['referees'][0]['name']
            else:
                return "Unknown"
        else:
            print(f"Failed to fetch data for Match ID {match_id}: {response.status_code}")
            return "Unknown"
    except Exception as e:
        print(f"Error fetching Match ID {match_id}: {e}")
        return "Unknown"

# Create a DataFrame for Match ID and Referee
match_ids = range(497410, 497790)  # Match IDs from 497410 to 497789
data = []

for match_id in match_ids:
    referee = fetch_referee_data(match_id)
    data.append({'Match ID': match_id, 'Referee': referee})
    time.sleep(10)  # To avoid hitting rate limits

# Convert to DataFrame
referee_df = pd.DataFrame(data)

# Save to CSV or display
referee_df.to_csv('referee_data.csv', index=False)
print(referee_df.head())


KeyboardInterrupt: 

Rabid API: Get Statistics

## FPL

In [194]:
import requests
import pandas as pd

# Fetch fixtures data from FPL API
url = "https://fantasy.premierleague.com/api/fixtures/"
response = requests.get(url)
fixtures = response.json()

# Extract relevant data
fdr_data = []
for match in fixtures:
    fdr_data.append({
        "gameweek": match.get("event"),
        "home_team_id": match.get("team_h"),
        "away_team_id": match.get("team_a"),
        "home_team_fdr": match.get("team_h_difficulty"),
        "away_team_fdr": match.get("team_a_difficulty")
    })

# Convert to DataFrame
fdr_df = pd.DataFrame(fdr_data)

# Map team IDs to names using the FPL static data API
teams_url = "https://fantasy.premierleague.com/api/bootstrap-static/"
teams_response = requests.get(teams_url)
teams = teams_response.json()["teams"]

team_map = {team["id"]: team["name"] for team in teams}

# Add team names to the DataFrame
fdr_df["home_team"] = fdr_df["home_team_id"].map(team_map)
fdr_df["away_team"] = fdr_df["away_team_id"].map(team_map)

# Drop team IDs for clarity
fdr_df.drop(columns=["home_team_id", "away_team_id"], inplace=True)

fdr_df.tail()

,gameweek,home_team_fdr,away_team_fdr,home_team,away_team
375,38.0,2,4,Newcastle,Everton
376,38.0,4,2,Nott'm Forest,Chelsea
377,38.0,5,2,Southampton,Arsenal
378,38.0,2,4,Spurs,Brighton
379,38.0,2,2,Wolves,Brentford


In [196]:
fdr_df.to_csv("fdr_data.csv")

In [201]:
import requests
import pandas as pd

# Fetch bootstrap-static data
bootstrap_url = "https://fantasy.premierleague.com/api/bootstrap-static/"
bootstrap_data = requests.get(bootstrap_url).json()

# Fetch fixtures data
fixtures_url = "https://fantasy.premierleague.com/api/fixtures/"
fixtures_data = requests.get(fixtures_url).json()

# Convert to DataFrames
players_df = pd.DataFrame(bootstrap_data['elements'])  # Player-level data
teams_df = pd.DataFrame(bootstrap_data['teams'])       # Team-level data
fixtures_df = pd.DataFrame(fixtures_data)              # Fixture data

# Save data for reuse
players_df.to_csv("players.csv", index=False)
teams_df.to_csv("teams.csv", index=False)
fixtures_df.to_csv("fixtures.csv", index=False)


In [203]:
players_df.head()

,can_transact,can_select,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,id,in_dreamteam,news,news_added,now_cost,photo,points_per_game,removed,second_name,selected_by_percent,special,squad_number,status,team,team_code,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,region,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text,expected_goals_per_90,saves_per_90,expected_assists_per_90,expected_goal_involvements_per_90,expected_goals_conceded_per_90,goals_conceded_per_90,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90
0,True,False,0.0,0.0,438098,0,0,-1,1,0,3,0.0,0.0,0,Fábio,0.0,1,False,Has joined Portuguese side FC Porto on loan fo...,2024-08-29T11:06:25.241953Z,54,438098.jpg,0.0,False,Ferreira Vieira,0.0,False,None,u,1,3,0,439,0,2758,9,0.0,0.0,Fábio Vieira,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.00,0.00,0.00,0.00,683,310,682,310,680,308,683,310,NaN,,NaN,,NaN,,0.00,0.0,0.00,0.00,0.00,0.00,137,82,674,304,682,310,636,278,0.00,0.00
1,True,True,100.0,100.0,205651,0,0,-2,2,0,4,1.8,1.8,1,Gabriel,0.8,2,False,,2024-08-25T11:00:05.312580Z,68,205651.jpg,0.8,False,Fernando de Jesus,0.9,False,None,a,1,3,9,62691,1006,229844,1852,0.1,1.3,G.Jesus,30.0,243,0,0,1,2,0,0,0,4,0,0,0,36,26.6,63.6,82.0,17.1,2,0.87,0.12,0.99,2.44,378,37,227,28,171,33,298,32,NaN,,NaN,,NaN,,0.32,0.0,0.04,0.36,0.90,0.74,32,15,303,38,459,49,213,37,0.74,0.37
2,True,True,75.0,75.0,226597,0,0,2,-2,2,2,3.6,3.6,0,Gabriel,3.8,3,True,Muscle injury - 75% chance of playing,2024-11-27T10:30:06.325759Z,62,226597.jpg,4.6,False,dos Santos Magalhães,23.1,False,None,d,1,3,60,2879039,34225,1909740,179139,0.6,9.7,Gabriel,30.0,1088,3,1,4,13,0,0,0,1,0,0,4,210,313.4,68.2,215.0,59.7,13,1.76,0.50,2.26,12.50,46,14,216,55,58,2,85,14,NaN,,NaN,,NaN,,0.15,0.0,0.04,0.19,1.03,1.08,66,3,76,17,31,4,13,5,1.08,0.33
3,True,True,100.0,100.0,219847,0,0,-1,1,1,4,4.2,4.2,5,Kai,3.2,4,False,,2024-11-07T09:00:07.112803Z,79,219847.jpg,4.2,False,Havertz,10.1,False,None,a,1,3,59,2162852,27738,2621576,28106,0.4,7.5,Havertz,80.0,1243,5,2,4,15,0,0,0,2,0,0,7,208,280.8,160.0,437.0,87.8,14,5.75,1.16,6.91,13.85,73,12,112,9,11,4,29,7,NaN,,NaN,,3.0,,0.42,0.0,0.08,0.50,1.00,1.09,13,5,88,18,37,14,37,10,1.01,0.29
4,True,False,0.0,0.0,463748,0,0,0,0,0,1,0.0,0.0,0,Karl,0.0,5,False,Loaned to Real Valladolid,2024-08-14T08:31:46.556082Z,40,463748.jpg,0.0,False,Hein,0.0,False,None,u,1,3,0,0,0,3968,50,0.0,0.0,Hein,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.00,0.00,0.00,0.00,511,58,491,37,455,27,513,58,NaN,,NaN,,NaN,,0.00,0.0,0.00,0.00,0.00,0.00,638,63,425,51,514,58,558,68,0.00,0.00


In [205]:
teams_df.head()

,code,draw,form,id,loss,name,played,points,position,short_name,strength,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
0,3,0,None,1,0,Arsenal,0,0,0,ARS,5,None,False,0,1350,1375,1360,1360,1340,1390,1
1,7,0,None,2,0,Aston Villa,0,0,0,AVL,3,None,False,0,1160,1220,1120,1140,1200,1300,2
2,91,0,None,3,0,Bournemouth,0,0,0,BOU,3,None,False,0,1100,1145,1070,1130,1130,1160,127
3,94,0,None,4,0,Brentford,0,0,0,BRE,3,None,False,0,1090,1155,1080,1120,1100,1190,130
4,36,0,None,5,0,Brighton,0,0,0,BHA,3,None,False,0,1100,1175,1070,1130,1130,1220,131


In [207]:
players_df.columns

Index(['can_transact', 'can_select', 'chance_of_playing_next_round',
       'chance_of_playing_this_round', 'code', 'cost_change_event',
       'cost_change_event_fall', 'cost_change_start', 'cost_change_start_fall',
       'dreamteam_count', 'element_type', 'ep_next', 'ep_this', 'event_points',
       'first_name', 'form', 'id', 'in_dreamteam', 'news', 'news_added',
       'now_cost', 'photo', 'points_per_game', 'removed', 'second_name',
       'selected_by_percent', 'special', 'squad_number', 'status', 'team',
       'team_code', 'total_points', 'transfers_in', 'transfers_in_event',
       'transfers_out', 'transfers_out_event', 'value_form', 'value_season',
       'web_name', 'region', 'minutes', 'goals_scored', 'assists',
       'clean_sheets', 'goals_conceded', 'own_goals', 'penalties_saved',
       'penalties_missed', 'yellow_cards', 'red_cards', 'saves', 'bonus',
       'bps', 'influence', 'creativity', 'threat', 'ict_index', 'starts',
       'expected_goals', 'expected_assists'

In [364]:
import requests
import pandas as pd

# Your API token
API_KEY = 'cadf4921727a4fc98eb9e50304e95e1f'
BASE_URL = 'https://api.football-data.org/v4/'

# Function to fetch data from the API
def fetch_data(endpoint, params=None):
    headers = {'X-Auth-Token': API_KEY}
    response = requests.get(BASE_URL + endpoint, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()
    elif response.status_code == 403:
        print("Access forbidden: Check if your token is valid for v4.")
    elif response.status_code == 429:
        print("Rate limit exceeded: Try again later.")
    else:
        print(f"Failed to fetch data: {response.status_code}")
        print(response.text)
    return None

# Fetch Premier League fixtures for the season
def get_premier_league_fixtures(season):
    fixtures_data = fetch_data('competitions/PL/matches', params={"season": season})
    if fixtures_data and 'matches' in fixtures_data:
        return fixtures_data['matches']
    else:
        print("No fixtures data available.")
        return []

# Simulate standings at the time of each fixture
def simulate_standings(fixtures):
    # Initialize standings
    standings = {}

    # Helper function to update standings
    def update_team_stats(team, scored, conceded, result):
        if team not in standings:
            standings[team] = {
                "Played": 0, "Won": 0, "Drawn": 0, "Lost": 0,
                "Goals Scored": 0, "Goals Conceded": 0, "Goal Difference": 0, "Points": 0
            }
        standings[team]["Played"] += 1
        standings[team]["Goals Scored"] += scored
        standings[team]["Goals Conceded"] += conceded
        standings[team]["Goal Difference"] += scored - conceded

        if result == "WIN":
            standings[team]["Won"] += 1
            standings[team]["Points"] += 3
        elif result == "DRAW":
            standings[team]["Drawn"] += 1
            standings[team]["Points"] += 1
        elif result == "LOSS":
            standings[team]["Lost"] += 1

    # Helper function to rank teams
    def rank_teams(standings):
        return sorted(
            standings.items(),
            key=lambda x: (
                -x[1]["Points"],         # Sort by Points (descending)
                -x[1]["Goal Difference"], # Then by Goal Difference (descending)
                x[0]                     # Finally by Team Name (alphabetical)
            )
        )

    # Process fixtures sequentially
    fixture_details = []
    for fixture in fixtures:
        home_team = fixture["homeTeam"]["name"]
        away_team = fixture["awayTeam"]["name"]
        home_goals = fixture["score"]["fullTime"]["home"]
        away_goals = fixture["score"]["fullTime"]["away"]
        date = fixture["utcDate"]

        # Determine results
        if home_goals is not None and away_goals is not None:
            if home_goals > away_goals:
                home_result, away_result = "WIN", "LOSS"
            elif home_goals < away_goals:
                home_result, away_result = "LOSS", "WIN"
            else:
                home_result, away_result = "DRAW", "DRAW"

            # Update standings for both teams
            update_team_stats(home_team, home_goals, away_goals, home_result)
            update_team_stats(away_team, away_goals, home_goals, away_result)

        # Rank teams after this fixture
        ranked_teams = rank_teams(standings)
        team_positions = {team: idx + 1 for idx, (team, stats) in enumerate(ranked_teams)}

        # Record fixture details with standings
        fixture_details.append({
            "Match ID": fixture["id"],
            "Date": date,
            "Home Team": home_team,
            "Away Team": away_team,
            "Home Score": home_goals,
            "Away Score": away_goals,
            "Winner": fixture["score"]["winner"],
            # Add standings at the time of this fixture
            "Home Position": team_positions.get(home_team),
            "Home Played": standings.get(home_team, {}).get("Played", 0),
            "Home Won": standings.get(home_team, {}).get("Won", 0),
            "Home Drawn": standings.get(home_team, {}).get("Drawn", 0),
            "Home Lost": standings.get(home_team, {}).get("Lost", 0),
            "Home Goals Scored": standings.get(home_team, {}).get("Goals Scored", 0),
            "Home Goals Conceded": standings.get(home_team, {}).get("Goals Conceded", 0),
            "Home Goal Difference": standings.get(home_team, {}).get("Goal Difference", 0),
            "Home Points": standings.get(home_team, {}).get("Points", 0),
            "Away Position": team_positions.get(away_team),
            "Away Played": standings.get(away_team, {}).get("Played", 0),
            "Away Won": standings.get(away_team, {}).get("Won", 0),
            "Away Drawn": standings.get(away_team, {}).get("Drawn", 0),
            "Away Lost": standings.get(away_team, {}).get("Lost", 0),
            "Away Goals Scored": standings.get(away_team, {}).get("Goals Scored", 0),
            "Away Goals Conceded": standings.get(away_team, {}).get("Goals Conceded", 0),
            "Away Goal Difference": standings.get(away_team, {}).get("Goal Difference", 0),
            "Away Points": standings.get(away_team, {}).get("Points", 0),
        })
    return pd.DataFrame(fixture_details)

# Main function to fetch and save data
def main():
    season = 2024  # Specify the season
    fixtures = get_premier_league_fixtures(season)

    if fixtures:
        detailed_fixtures = simulate_standings(fixtures)
        # Save to CSV
        csv_file = f"premier_league_historical_fixtures_{season}_{season+1}.csv"
        detailed_fixtures.to_csv(csv_file, index=False)
        print(f"Historical fixtures data saved to {csv_file}")
        print(detailed_fixtures.head())
    else:
        print("Failed to fetch fixtures data.")

# Run the script
main()


Historical fixtures data saved to premier_league_historical_fixtures_2024_2025.csv
   Match ID                  Date             Home Team  \
0    497410  2024-08-16T19:00:00Z  Manchester United FC   
1    497411  2024-08-17T11:30:00Z       Ipswich Town FC   
2    497412  2024-08-17T14:00:00Z            Arsenal FC   
3    497413  2024-08-17T14:00:00Z            Everton FC   
4    497414  2024-08-17T14:00:00Z   Newcastle United FC   

                    Away Team  Home Score  Away Score     Winner  \
0                   Fulham FC         1.0         0.0  HOME_TEAM   
1                Liverpool FC         0.0         2.0  AWAY_TEAM   
2  Wolverhampton Wanderers FC         2.0         0.0  HOME_TEAM   
3   Brighton & Hove Albion FC         0.0         3.0  AWAY_TEAM   
4              Southampton FC         1.0         0.0  HOME_TEAM   

   Home Position  Home Played  Home Won  Home Drawn  Home Lost  \
0              1            1         1           0          0   
1              4     

In [366]:
detailed_fixtures = pd.read_csv("premier_league_historical_fixtures_2024_2025.csv")

In [368]:
detailed_fixtures.head()

,Match ID,Date,Home Team,Away Team,Home Score,Away Score,Winner,Home Position,Home Played,Home Won,Home Drawn,Home Lost,Home Goals Scored,Home Goals Conceded,Home Goal Difference,Home Points,Away Position,Away Played,Away Won,Away Drawn,Away Lost,Away Goals Scored,Away Goals Conceded,Away Goal Difference,Away Points
0,497410,2024-08-16T19:00:00Z,Manchester United FC,Fulham FC,1.0,0.0,HOME_TEAM,1,1,1,0,0,1,0,1,3,2,1,0,0,1,0,1,-1,0
1,497411,2024-08-17T11:30:00Z,Ipswich Town FC,Liverpool FC,0.0,2.0,AWAY_TEAM,4,1,0,0,1,0,2,-2,0,1,1,1,0,0,2,0,2,3
2,497412,2024-08-17T14:00:00Z,Arsenal FC,Wolverhampton Wanderers FC,2.0,0.0,HOME_TEAM,1,1,1,0,0,2,0,2,3,6,1,0,0,1,0,2,-2,0
3,497413,2024-08-17T14:00:00Z,Everton FC,Brighton & Hove Albion FC,0.0,3.0,AWAY_TEAM,8,1,0,0,1,0,3,-3,0,1,1,1,0,0,3,0,3,3
4,497414,2024-08-17T14:00:00Z,Newcastle United FC,Southampton FC,1.0,0.0,HOME_TEAM,5,1,1,0,0,1,0,1,3,7,1,0,0,1,0,1,-1,0
